<a href="https://colab.research.google.com/github/Nicolas-BZRD/Multi-Video-Temporal-Synchronization/blob/main/Siamese_CNN_model_identical_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Siamese CNN - identical image

In [4]:
import os
import random
from tqdm import tqdm

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras import models, layers

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Dataset

dataset: https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip

Pre-processing of data

In [6]:
# data_dir = "/content/drive/MyDrive/ISIA Lab/Dataset/VG"
# listImages = os.listdir(data_dir)

# for image in tqdm(listImages):
#   try:
#     img = cv2.imread(f'/content/drive/MyDrive/ISIA Lab/Dataset/VG/{image}')
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img = cv2.resize(img, dsize=(260, 260), interpolation=cv2.INTER_CUBIC)
#     cv2.imwrite(f'/content/drive/MyDrive/ISIA Lab/Dataset/VG Gray/{image}', img)
#   except:
#     print(f'(error: {image}')

Tool functions:

In [7]:
def randomPairSameImageCroped(npArr):
  height, width = npArr.shape

  randHeight = random.randint(0, int(height*0.2))
  randWidth = random.randint(0, int(width*0.2))

  randReturn = random.randint(0,3)

  if randReturn == 0:
    return (
      npArr[0:(height-randHeight),0:(width-randWidth)],
      npArr[0:(height-randHeight),0:(width-randWidth)]
    )
  elif randReturn == 1:
    return (
      npArr[0:(height-randHeight),0:(width-randWidth)],
      npArr[randHeight:, 0:(width-randWidth)]
    )
  elif randReturn == 2:
    return (
      npArr[0:(height-randHeight),0:(width-randWidth)],
      npArr[randHeight:, randWidth:]
    )
  else:
    return (
      npArr[0:(height-randHeight),0:(width-randWidth)],
      npArr[0:(height-randHeight), randWidth:]
    )



In [8]:
def randomPairImage(list):
  l = r = ""
  while(l == r):
    l = random.choice(list)
    r = random.choice(list)
  return l,r

Open dataset

In [9]:
data_dir = "/content/drive/MyDrive/ISIA Lab/Dataset/VG Gray"
listImages = os.listdir(data_dir)
random.shuffle(listImages)

dataset = []
len(listImages)

30929

Creating a pair of correct cropped image:

In [10]:
for item in tqdm(listImages):
  img = cv2.imread(f'{data_dir}/{item}', cv2.IMREAD_GRAYSCALE)
  pair = randomPairSameImageCroped(img)
  dataset.append([pair, 1])

100%|██████████| 30929/30929 [02:45<00:00, 187.31it/s]


Creation of bad couple of images (same quantity as correct couple of images):

In [11]:
size = len(dataset)
for i in tqdm(range(0,size)):
  l, r = randomPairImage(listImages)
  l = cv2.imread(f'{data_dir}/{l}', cv2.IMREAD_GRAYSCALE)
  r = cv2.imread(f'{data_dir}/{r}', cv2.IMREAD_GRAYSCALE)
  dataset.append([(l,r), 0])

100%|██████████| 30929/30929 [04:28<00:00, 115.02it/s]


##Model

In [12]:
IMG_SHAPE = (224, 224, 1)

BASE_OUTPUT = "/content/drive/MyDrive/ISIA Lab/Siamese CNN"
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "siamese_model"])

In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=IMG_SHAPE))
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Conv2D(48, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.15))

model.add(layers.Conv2D(48, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(48, activation='relu'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 48)        13872     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 48)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 36, 36, 48)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 48)       